# About

_TODO_

For more info on setting up Accumulo, see the "D4M Accumulo Demo".

_TODO_ Don't forget to add Graphulo jars to your server-side Accumulo files! More info here: _TODO_ 

# Accumulo

In [8]:
using D4M

┌ Info: Precompiling D4M [ca196bdc-a701-11e8-3d50-3b5cc8577617]
└ @ Base loading.jl:1192


Database capabilities loaded!
D4M loaded!


In [9]:
dbinit()

In [27]:
# DB = dbsetup("uno", "AccumuloConfig.jl")
DB = dbsetup("uno", "localhost:2181", "root", "secret")

22 Jul 2019 22:38:56,509 WARN - ClientConfiguration.loadFromSearchPath(227) -  Found no client.conf in default paths. Using default client configuration values.


D4M.DBserver("uno", "localhost:2181", "root", "secret", "BigTableLike", JavaCall.JavaObject{Symbol("edu.mit.ll.graphulo.MatlabGraphulo")}(Ptr{Nothing} @0x000000000417b7d0))

In [ ]:
deleteall(DB)

In [4]:
A = DB["mytable_A"]
AT = DB["mytable_AT"]
B = DB["mytable_B"]


ls(DB)

Creating mytable_A in uno
Creating mytable_AT in uno
Creating mytable_B in uno


19-element Array{SubString{String},1}:
 "accumulo.metadata"                  
 "accumulo.replication"               
 "accumulo.root"                      
 "my_tweets_user_share_hashtag_deg"   
 "mytable_A"                          
 "mytable_AT"                         
 "mytable_B"                          
 "t_Tedge"                            
 "t_TedgeDeg"                         
 "t_TedgeT"                           
 "t_hashtag_user"                     
 "t_user_hashtag"                     
 "t_user_share_hashtag"               
 "t_user_share_hashtag_deg"           
 "t_user_share_hashtag_nodiag"        
 "t_user_share_hashtag_nodiag_deg"    
 "t_user_share_hashtag_nodiag_jaccard"
 "t_wordword"                         
 "trace"                              

In [5]:
#row = ["r1", "r1", "r2", "r2"]
row = ["1", "1", "2", "2"]
column = ["1","2","1","2"]
values = [1,2,3,4]

AssocA = Assoc(row,column,values)
put(A, AssocA, clear=true)
printFull(A[:,:])

AssocAT = transpose(AssocA)
put(AT, AssocAT, clear=true)
printFull(AT[:,:])

row = ["1", "1", "2", "2"]
column = ["1","2","1","2"]
values = [5,6,7,8]

AssocB = Assoc(row,column,values)
put(B, AssocB, clear=true)
printFull(B[:,:])

3×3 Array{Union{AbstractString, Number},2}:
 ""   "1"  "2"
 "1"  "1"  "2"
 "2"  "3"  "4"

3×3 Array{Union{AbstractString, Number},2}:
 ""   "1"  "2"
 "1"  "1"  "3"
 "2"  "2"  "4"

3×3 Array{Union{AbstractString, Number},2}:
 ""   "1"  "2"
 "1"  "5"  "6"
 "2"  "7"  "8"

(for some reason, printFull prints the entries as strings... however, in Accumulo, they're still numbers and can be operated on, as we'll soon see)

# Table Multiplication

_TODO expand on this_
To multiply A and B, we pass in <b><u>A'</b></u> and B

In [25]:
P = tablemult(AT, B, "mytable_P", clear=true)
printFull(P)

28 Jun 2019 20:09:59,843 DEBUG - Graphulo.TwoTable(770) -   :%00; [1] -> 8 entries processed


MethodError: MethodError: no method matching getindex(::D4M.DBtable, ::Colon)
Closest candidates are:
  getindex(::Union{DBtable, DBtablePair}, ::Union{Colon, AbstractString, StartsWith, Array}, !Matched::Union{Colon, AbstractString, StartsWith, Array}) at /home/nathan/Documents/UROP/D4M.jl/src/DBtable.jl:122
  getindex(::Union{DBtable, DBtablePair}) at /home/nathan/Documents/UROP/D4M.jl/src/DBtable.jl:154
  getindex(::Union{DBtable, DBtablePair}, !Matched::AbstractString, !Matched::AbstractString) at /home/nathan/Documents/UROP/D4M.jl/src/DBtable.jl:89

Let's check this...

In [9]:
printFull(AssocA * AssocB)

3×3 Array{Union{AbstractString, Number},2}:
 ""     "1"    "2"
 "1"  19     22   
 "2"  43     50   

We can add filters too. They are added as extra arguments to `tablemult`, and are formatted like D4M query strings (what you might put in a normal associative array filter). For example, here are some simple row filters:

In [27]:
Pfilter1 = tablemult(AT, B, "mytable_Pfilter1", rowfilter="1,", clear=true)
printFull(Pfilter1)

3×3 Array{Union{AbstractString, Number},2}:
 ""   "1"   "2" 
 "1"  "5"   "6" 
 "2"  "15"  "18"

28 Jun 2019 20:10:34,833 DEBUG - Graphulo.TwoTable(770) -  1 :%00; [1] -> 4 entries processed


Table multiplication first applies filters to AT, transposes it, and multiplies. We can check by using native D4M:

In [11]:
printFull(transpose(AssocAT["1,",:]) * AssocB["1,",:])

3×3 Array{Union{AbstractString, Number},2}:
 ""     "1"    "2"
 "1"   5      6   
 "2"  15     18   

This is equivalent to running a column filter on the original matrix:

In [12]:
printFull(AssocA[:,1] * AssocB[1,:])

3×3 Array{Union{AbstractString, Number},2}:
 ""     "1"    "2"
 "1"   5      6   
 "2"  15     18   

As for column filters in `tablemult`:

In [13]:
Pfilter2 = tablemult(AT, B, "mytable_Pfilter2", colfilterAT="1,", clear=true)
printFull(Pfilter2[:,:])
printFull(transpose(AssocAT[:,"1,"]) * AssocB[:,:]) # to check

2×3 Array{Union{AbstractString, Number},2}:
 ""   "1"   "2" 
 "1"  "19"  "22"

2×3 Array{Union{AbstractString, Number},2}:
 ""     "1"    "2"
 "1"  19     22   

28 Jun 2019 13:49:11,335 DEBUG - Graphulo.TwoTable(770) -   :%00; [1] -> 4 entries processed


In [14]:
Pfilter3 = tablemult(AT, B, "mytable_Pfilter3", colfilterB="1,", clear=true)
printFull(Pfilter3[:,:])
printFull(transpose(AssocAT[:,:]) * AssocB[:,"1,"]) # to check

3×2 Array{Union{AbstractString, Number},2}:
 ""   "1" 
 "1"  "19"
 "2"  "43"

3×2 Array{Union{AbstractString, Number},2}:
 ""     "1"
 "1"  19   
 "2"  43   

28 Jun 2019 13:49:13,167 DEBUG - Graphulo.TwoTable(770) -   :%00; [1] -> 4 entries processed


Now, let's try some more extensive filters, with a bigger example. First, we create an associative array, and square it:

In [15]:
row = "a,a,a,a,a,a,a,aa,aaa,b,bb,bbb,a,aa,aaa,b,bb,bbb,aaa,b,bb,bbb,"
column = "a,aa,aaa,b,bb,bbb,a,a,a,a,a,a,a,aa,aaa,b,bb,bbb,aa,aa,aa,aa,"
values = "a-a,a-aa,a-aaa,a-b,a-bb,a-bbb,a-a,aa-a,aaa-a,b-a,bb-a,bbb-a,a-a,aa-aa,aaa-aaa,b-b,bb-bb,bbb-bbb,a,a,a,a,"
AssocD = logical(Assoc(row,column,values))
printFull(AssocD)
AssocDT = transpose(AssocD)

D = DB["mytable_D"]
put(D, AssocD, clear=true)
printFull(D[:,:])

DT = DB["mytable_DT"]
put(DT, AssocDT, clear=true)
printFull(DT[:,:])

7×7 Array{Union{AbstractString, Number},2}:
 ""      "a"   "aa"   "aaa"   "b"   "bb"   "bbb"
 "a"    1     1      1       1     1      1     
 "aa"   1     1      0       0     0      0     
 "aaa"  1     1      1       0     0      0     
 "b"    1     1      0       1     0      0     
 "bb"   1     1      0       0     1      0     
 "bbb"  1     1      0       0     0      1     

Creating mytable_D in uno


7×7 Array{Union{AbstractString, Number},2}:
 ""     "a"  "aa"  "aaa"  "b"  "bb"  "bbb"
 "a"    "1"  "1"   "1"    "1"  "1"   "1"  
 "aa"   "1"  "1"   ""     ""   ""    ""   
 "aaa"  "1"  "1"   "1"    ""   ""    ""   
 "b"    "1"  "1"   ""     "1"  ""    ""   
 "bb"   "1"  "1"   ""     ""   "1"   ""   
 "bbb"  "1"  "1"   ""     ""   ""    "1"  

Creating mytable_DT in uno


7×7 Array{Union{AbstractString, Number},2}:
 ""     "a"  "aa"  "aaa"  "b"  "bb"  "bbb"
 "a"    "1"  "1"   "1"    "1"  "1"   "1"  
 "aa"   "1"  "1"   "1"    "1"  "1"   "1"  
 "aaa"  "1"  ""    "1"    ""   ""    ""   
 "b"    "1"  ""    ""     "1"  ""    ""   
 "bb"   "1"  ""    ""     ""   "1"   ""   
 "bbb"  "1"  ""    ""     ""   ""    "1"  

In [16]:
D2 = tablemult(DT, D, "mytable_D2", clear=true)
printFull(D2[:,:])
printFull(AssocD * AssocD)

7×7 Array{Union{AbstractString, Number},2}:
 ""     "a"  "aa"  "aaa"  "b"  "bb"  "bbb"
 "a"    "6"  "6"   "2"    "2"  "2"   "2"  
 "aa"   "2"  "2"   "1"    "1"  "1"   "1"  
 "aaa"  "3"  "3"   "2"    "1"  "1"   "1"  
 "b"    "3"  "3"   "1"    "2"  "1"   "1"  
 "bb"   "3"  "3"   "1"    "1"  "2"   "1"  
 "bbb"  "3"  "3"   "1"    "1"  "1"   "2"  

7×7 Array{Union{AbstractString, Number},2}:
 ""      "a"   "aa"   "aaa"   "b"   "bb"   "bbb"
 "a"    6     6      2       2     2      2     
 "aa"   2     2      1       1     1      1     
 "aaa"  3     3      2       1     1      1     
 "b"    3     3      1       2     1      1     
 "bb"   3     3      1       1     2      1     
 "bbb"  3     3      1       1     1      2     

28 Jun 2019 13:49:24,013 DEBUG - Graphulo.TwoTable(770) -   :%00; [1] -> 72 entries processed


Let's try some filters:

Multiplying only a few rows and columns:

In [17]:
D2f1 = tablemult(DT, D, "mytable_D2f1", rowfilter = "a,b,", colfilterAT = "a,bb,", clear=true)
printFull(D2f1[:,:])
printFull(transpose(AssocDT["a,b,","a,bb,"]) * AssocD["a,b,",:])

3×7 Array{Union{AbstractString, Number},2}:
 ""    "a"  "aa"  "aaa"  "b"  "bb"  "bbb"
 "a"   "2"  "2"   "1"    "2"  "1"   "1"  
 "bb"  "1"  "1"   "1"    "1"  "1"   "1"  

28 Jun 2019 13:49:25,609 DEBUG - Graphulo.TwoTable(770) -  b :%00; [2] -> 15 entries processed


3×7 Array{Union{AbstractString, Number},2}:
 ""     "a"   "aa"   "aaa"   "b"   "bb"   "bbb"
 "a"   2     2      1       2     1      1     
 "bb"  1     1      1       1     1      1     

Multiplying ranges of rows and columns:

In [18]:
D2f2 = tablemult(DT, D, "mytable_D2f2", rowfilter = "a,:,b,", colfilterB = "a,:,aaa,", clear=true)
printFull(D2f2[:,:])
printFull(transpose(AssocDT["a,:,b,",:]) * AssocD["a,:,b,","a,:,aaa,"])

7×4 Array{Union{AbstractString, Number},2}:
 ""     "a"  "aa"  "aaa"
 "a"    "4"  "4"   "2"  
 "aa"   "2"  "2"   "1"  
 "aaa"  "3"  "3"   "2"  
 "b"    "3"  "3"   "1"  
 "bb"   "2"  "2"   "1"  
 "bbb"  "2"  "2"   "1"  

7×4 Array{Union{AbstractString, Number},2}:
 ""      "a"   "aa"   "aaa"
 "a"    4     4      2     
 "aa"   2     2      1     
 "aaa"  3     3      2     
 "b"    3     3      1     
 "bb"   2     2      1     
 "bbb"  2     2      1     

28 Jun 2019 13:49:27,390 DEBUG - Graphulo.TwoTable(770) -  a :%00; [1] -> 40 entries processed


Multiplying rows and columns that start with certain strings:

In [20]:
D2f3 = tablemult(DT, D, "mytable_D2f3", colfilterAT = StartsWith("aa"), clear=true)
printFull(D2f3[:,:])
printFull(transpose(AssocDT[:,StartsWith("aa")]) * AssocD[:,:])

3×7 Array{Union{AbstractString, Number},2}:
 ""     "a"  "aa"  "aaa"  "b"  "bb"  "bbb"
 "aa"   "2"  "2"   "1"    "1"  "1"   "1"  
 "aaa"  "3"  "3"   "2"    "1"  "1"   "1"  

3×7 Array{Union{AbstractString, Number},2}:
 ""      "a"   "aa"   "aaa"   "b"   "bb"   "bbb"
 "aa"   2     2      1       1     1      1     
 "aaa"  3     3      2       1     1      1     

28 Jun 2019 13:49:43,318 DEBUG - Graphulo.TwoTable(770) -   :%00; [1] -> 19 entries processed


We can try some other forms of queries too:

In [1]:
StringArray = Array{T} where T <: AbstractString
NumericArray = Array{T} where T <: Number
UnionArray = Array{T} where T <: Union{AbstractString,Number}

Array{T,N} where N where T<:Union{AbstractString, Number}

In [22]:
isa([1,2], NumericArray)

true

In [23]:
row = "a,a,a,a,a,a,a,aa,aaa,b,bb,bbb,a,aa,aaa,b,bb,bbb,aaa,b,bb,bbb,"
column = "a,aa,aaa,b,bb,bbb,a,a,a,a,a,a,a,aa,aaa,b,bb,bbb,aa,aa,aa,aa,"
values = "a-a,a-aa,a-aaa,a-b,a-bb,a-bbb,a-a,aa-a,aaa-a,b-a,bb-a,bbb-a,a-a,aa-aa,aaa-aaa,b-b,bb-bb,bbb-bbb,a,a,a,a,"
AssocD = logical(Assoc(row,column,values))
printFull(AssocD)
printFull(AssocD[StartsWith("aa"), :])

7×7 Array{Union{AbstractString, Number},2}:
 ""      "a"   "aa"   "aaa"   "b"   "bb"   "bbb"
 "a"    1     1      1       1     1      1     
 "aa"   1     1      0       0     0      0     
 "aaa"  1     1      1       0     0      0     
 "b"    1     1      0       1     0      0     
 "bb"   1     1      0       0     1      0     
 "bbb"  1     1      0       0     0      1     

3×4 Array{Union{AbstractString, Number},2}:
 ""      "a"   "aa"   "aaa"
 "aa"   1     1      0     
 "aaa"  1     1      1     

# Degree Tables

# Breadth-First Search

_TODO explain_

In [11]:
E = ReadCSV("Edge.csv")
Ev = logical( E[:, StartsWith("V,")] )
printFull(Ev)

20×21 Array{Union{AbstractString, Number},2}:
 ""     "V01"   "V02"   "V03"   "V04"  …   "V17"   "V18"   "V19"   "V20"
 "B1"  1       1       1       0          0       0       0       0     
 "B2"  0       0       0       1          0       0       0       0     
 "G1"  1       1       1       0          0       0       0       0     
 "G2"  0       0       0       1          0       0       0       0     
 "O1"  1       1       1       0       …  0       0       0       0     
 "O2"  1       1       1       0          0       0       0       0     
 "O3"  0       0       0       1          0       0       0       0     
 "O4"  0       0       0       1          0       0       0       0     
 "O5"  0       1       0       0          0       0       0       1     
 "P1"  1       1       1       0       …  0       0       0       0     
 "P2"  0       0       0       1          0       0       0       0     
 "P3"  0       0       0       0          0       0       0       0     
 "P4"

In [12]:
edgematrix = DB["edgematrix"]
put(edgematrix, Ev, clear=true)

In [6]:
Av = sqIn(Ev)
printFull(Av)

21×21 Array{Union{AbstractString, Number},2}:
 ""      "V01"   "V02"   "V03"   "V04"  …   "V17"   "V18"   "V19"   "V20"
 "V01"  6       6       6       0          0       0       0       0     
 "V02"  6       9       6       0          0       0       0       1     
 "V03"  6       6       6       0          0       0       0       0     
 "V04"  0       0       0       6          0       0       0       0     
 "V05"  0       0       0       6       …  0       1       0       0     
 "V06"  0       1       0       6          0       0       0       1     
 "V07"  0       0       0       6          0       0       1       0     
 "V08"  0       0       0       6          0       0       0       0     
 "V09"  0       1       0       0          0       0       0       1     
 "V10"  0       1       0       0       …  0       0       0       0     
 "V11"  0       1       0       0          0       0       0       1     
 "V12"  0       1       0       0          0       0       0      

In [7]:
adjmatrix = DB["adjmatrix"]
put(adjmatrix, Av, clear=true)

In [8]:
adjbfs(adjmatrix,"V02,V20,",2,"bfsresult","bfsresultT", 0, 6)

22 Jul 2019 12:24:37,502 DEBUG - Graphulo.OneTable(987) -  V20 :%00; [2] -> 6 entries processed
22 Jul 2019 12:24:37,555 DEBUG - Graphulo.OneTable(987) -  V20 :%00; [6] -> 12 entries processed


"V20,V11,V02,V16,V06,V09,"

In [9]:
bfsresult = DB["bfsresult"]
printFull(bfsresult)

3×7 Array{Union{AbstractString, Number},2}:
 ""     "V02"  "V06"  "V09"  "V11"  "V16"  "V20"
 "V09"  "2"    "2"    "2"    "2"    "2"    "2"  
 "V20"  "4"    "4"    "4"    "4"    "4"    "4"  

In [16]:
edgebfs(edgematrix, "V02,",1,"edgebfsresult3","edgebfsresultT3", 0, 2^31-1)

22 Jul 2019 22:33:17,420 DEBUG - Graphulo.EdgeBFS(1445) -  fetchColumn :	
22 Jul 2019 22:33:17,421 DEBUG - EdgeBFSReducer.parseOptions(34) -  inColumnPrefixes: ,


"V10,V20,V01,V12,V11,V03,V02,V13,V16,V06,V09,"